In [1]:
import os
from ast import literal_eval
from tqdm import tqdm
import pandas as pd
import numpy as np
np.random.seed(1)
import warnings
warnings.filterwarnings('ignore')
import pickle
from rdkit.Chem import MACCSkeys
from rdkit.Chem import MolFromSmiles
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from sklearn.metrics import roc_auc_score
# 每个dataset每个靶点，加载每一个算法的最好指纹和对应超参，训练全部数据，然后用外部验证集来测试
# 验证集是训练集的子集


#文件路径准备
basePath = os.getcwd()
resultPath = basePath+'/results/'
training_path = basePath+'/training_data/'
external_path = basePath+'/external_data/'


datasets_list = os.listdir(external_path)
algorithm_list = ['KNN','NB','RF','XGBoost','SVC']
FingerPrint_list = ['ecfp6','ecfp6fcfp6','ecfp6fcfp6MACCS','ecfp6MACCS','fcfp6','fcfp6MACCS','MACCS']

In [ ]:
# 记录最佳指纹并保存

best_results = pd.DataFrame(columns=['dataset', 'target', 'model', 'best_prauc_FP','best_prauc_params','best_prauc_score', 'best_rocauc_FP','best_rocauc_params','best_rocauc_score'])

for dataset in datasets_list:
    for target in os.listdir(external_path + dataset):
        for model in algorithm_list:
            results_df = pd.DataFrame()
            for fp in FingerPrint_list:
                result = pd.read_csv(resultPath + fp + '/' + fp + '_' + model + '_' + dataset.split('_',1)[1] + '_rocmean.csv')
                target_row = result.loc[result['targets'] == target.split('.')[0]]
                results_df = pd.concat([results_df,target_row], ignore_index=True)
            # print(results_df)
            prauc_max = results_df['prauc_score'].idxmax()
            rocauc_max = results_df['rocauc_score'].idxmax()
            best_results.loc[len(best_results.index)] = [dataset.split('_',1)[1], target.split('.')[0], model, 
            FingerPrint_list[prauc_max], results_df.at[prauc_max, 'prauc_params'], results_df.at[prauc_max, 'prauc_score'],
            FingerPrint_list[rocauc_max], results_df.at[rocauc_max, 'rocauc_params'], results_df.at[rocauc_max, 'rocauc_score']]


best_results.to_excel('training_list.xlsx', index=False)

In [2]:
# SMILES to fingerprint feature conversion / Different combinations of fingerprint features
def batchECFP(smiles, FingerPrint, radius=3, nBits=2048):
    smiles = np.array(smiles)
    n = len(smiles)
    fingerprints_0 = np.zeros((n, nBits), dtype=int)
    fingerprints_1 = np.zeros((n, nBits), dtype=int)
    MACCSArray = []
    for i in range(n):
        mol = MolFromSmiles(smiles[i])
        # ecfp6
        fp = GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)
        # fcfp6
        fp_1 = GetMorganFingerprintAsBitVect(
            mol, radius, nBits=nBits, useFeatures=True)
        # MACCS
        MACCSArray.append(MACCSkeys.GenMACCSKeys(mol))
        fingerprints_0[i] = np.array(list(fp.ToBitString()))
        fingerprints_1[i] = np.array(list(fp_1.ToBitString()))
    fingerprints_2 = np.array(MACCSArray)
    fingerprints = np.hstack((fingerprints_0, fingerprints_1, fingerprints_2))
    fingerprints_3 = np.hstack((fingerprints_0, fingerprints_1))
    fingerprints_4 = np.hstack((fingerprints_0, fingerprints_2))
    fingerprints_5 = np.hstack((fingerprints_1, fingerprints_2))
    if FingerPrint == "ecfp6fcfp6MACCS":
        fingerprints_out = fingerprints
    elif FingerPrint == "ecfp6":
        fingerprints_out = fingerprints_0
    elif FingerPrint == "fcfp6":
        fingerprints_out = fingerprints_1
    elif FingerPrint == "MACCS":
        fingerprints_out = fingerprints_2
    elif FingerPrint == "ecfp6fcfp6":
        fingerprints_out = fingerprints_3
    elif FingerPrint== "ecfp6MACCS":
        fingerprints_out = fingerprints_4
    elif FingerPrint == "fcfp6MACCS":
        fingerprints_out = fingerprints_5

    return fingerprints_out

def run_ext(model,X_train,y_train,X_test,y_test,algorithm,dataset,target,FP):
    model.fit(X_train,y_train)
    # pkl_filename = external_path+algorithm+'_'+dataset+'_'+target+'_'+FP+".pkl"
    # with open(pkl_filename, 'wb') as file:
    #     pickle.dump(model, file)
    y_pred_proba = model.predict_proba(X_test)
    rocauc_score = roc_auc_score(y_test, y_pred_proba[:, 1])
    return rocauc_score



training_list_df = pd.read_excel('training_list.xlsx', header=0,index_col=False)

In [3]:
#更换算法时修改
from sklearn.neighbors import KNeighborsClassifier
algorithm = 'KNN'

filtered_df = training_list_df[training_list_df['model'] == algorithm]

for i in tqdm(filtered_df.index, total=len(filtered_df)):

    model = KNeighborsClassifier(**literal_eval(filtered_df.at[i, 'best_rocauc_params']))

    dataset = filtered_df.at[i, 'dataset']
    target = filtered_df.at[i, 'target']
    FP = filtered_df.at[i, 'best_rocauc_FP']

    train_data = pd.read_csv(training_path+dataset+'/'+target+'.csv', header=0,index_col=False)
    X_train = batchECFP(train_data.iloc[:,2],FP)
    y_train = train_data.iloc[:,4]
    test_data = pd.read_csv(external_path+'ex_'+dataset+'/'+target+'.csv', header=0,index_col=False)
    X_test = batchECFP(test_data.iloc[:,1],FP)
    y_test = test_data.iloc[:,0]
    try:
        roc = run_ext(model,X_train,y_train,X_test,y_test,algorithm,dataset,target,FP)
    except Exception as e:
        roc = e
        print(e)
    filtered_df.at[i, 'external_roc'] = roc

filtered_df.to_excel(external_path+algorithm+'_ex_roc.xlsx',index=False)

  2%|▏         | 10/462 [00:06<04:38,  1.62it/s]

Only one class present in y_true. ROC AUC score is not defined in that case.


  3%|▎         | 13/462 [00:07<04:10,  1.79it/s]

Only one class present in y_true. ROC AUC score is not defined in that case.


  3%|▎         | 15/462 [00:09<04:39,  1.60it/s]

...

 98%|█████████▊| 452/462 [03:17<00:04,  2.21it/s]

Only one class present in y_true. ROC AUC score is not defined in that case.


 99%|█████████▉| 457/462 [03:20<00:02,  1.72it/s]

Only one class present in y_true. ROC AUC score is not defined in that case.


100%|██████████| 462/462 [03:23<00:00,  2.27it/s]


In [4]:
#更换算法时修改
from sklearn.ensemble import RandomForestClassifier
algorithm = 'RF'

filtered_df = training_list_df[training_list_df['model'] == algorithm]

for i in tqdm(filtered_df.index, total=len(filtered_df)):

    model = RandomForestClassifier(**literal_eval(filtered_df.at[i, 'best_rocauc_params']))
    
    dataset = filtered_df.at[i, 'dataset']
    target = filtered_df.at[i, 'target']
    FP = filtered_df.at[i, 'best_rocauc_FP']

    train_data = pd.read_csv(training_path+dataset+'/'+target+'.csv', header=0,index_col=False)
    X_train = batchECFP(train_data.iloc[:,2],FP)
    y_train = train_data.iloc[:,4]
    test_data = pd.read_csv(external_path+'ex_'+dataset+'/'+target+'.csv', header=0,index_col=False)
    X_test = batchECFP(test_data.iloc[:,1],FP)
    y_test = test_data.iloc[:,0]
    try:
        roc = run_ext(model,X_train,y_train,X_test,y_test,algorithm,dataset,target,FP)
    except Exception as e:
        roc = e
        print(e)
    filtered_df.at[i, 'external_roc'] = roc

filtered_df.to_excel(external_path+algorithm+'_ex_roc.xlsx',index=False)

  2%|▏         | 10/462 [00:07<05:14,  1.44it/s]

Only one class present in y_true. ROC AUC score is not defined in that case.


  3%|▎         | 13/462 [00:08<04:23,  1.70it/s]

Only one class present in y_true. ROC AUC score is not defined in that case.


  3%|▎         | 15/462 [00:10<05:50,  1.28it/s]

...

 98%|█████████▊| 452/462 [05:00<00:07,  1.37it/s]

Only one class present in y_true. ROC AUC score is not defined in that case.


 99%|█████████▉| 457/462 [05:06<00:05,  1.04s/it]

Only one class present in y_true. ROC AUC score is not defined in that case.


100%|██████████| 462/462 [05:10<00:00,  1.49it/s]


In [5]:
#更换算法时修改
from sklearn.naive_bayes import BernoulliNB
algorithm = 'NB'

filtered_df = training_list_df[training_list_df['model'] == algorithm]

for i in tqdm(filtered_df.index, total=len(filtered_df)):

    model = BernoulliNB(**literal_eval(filtered_df.at[i, 'best_rocauc_params']))
    
    dataset = filtered_df.at[i, 'dataset']
    target = filtered_df.at[i, 'target']
    FP = filtered_df.at[i, 'best_rocauc_FP']

    train_data = pd.read_csv(training_path+dataset+'/'+target+'.csv', header=0,index_col=False)
    X_train = batchECFP(train_data.iloc[:,2],FP)
    y_train = train_data.iloc[:,4]
    test_data = pd.read_csv(external_path+'ex_'+dataset+'/'+target+'.csv', header=0,index_col=False)
    X_test = batchECFP(test_data.iloc[:,1],FP)
    y_test = test_data.iloc[:,0]
    try:
        roc = run_ext(model,X_train,y_train,X_test,y_test,algorithm,dataset,target,FP)
    except Exception as e:
        roc = e
        print(e)
    filtered_df.at[i, 'external_roc'] = roc

filtered_df.to_excel(external_path+algorithm+'_ex_roc.xlsx',index=False)

  2%|▏         | 10/462 [00:07<05:20,  1.41it/s]

Only one class present in y_true. ROC AUC score is not defined in that case.


  3%|▎         | 13/462 [00:08<04:31,  1.65it/s]

Only one class present in y_true. ROC AUC score is not defined in that case.


  3%|▎         | 15/462 [00:10<04:53,  1.52it/s]

...

 98%|█████████▊| 452/462 [03:35<00:04,  2.03it/s]

Only one class present in y_true. ROC AUC score is not defined in that case.


 99%|█████████▉| 457/462 [03:38<00:03,  1.58it/s]

Only one class present in y_true. ROC AUC score is not defined in that case.


100%|██████████| 462/462 [03:42<00:00,  2.08it/s]


In [6]:
#更换算法时修改
from sklearn.svm import SVC
algorithm = 'SVC'

filtered_df = training_list_df[training_list_df['model'] == algorithm]

for i in tqdm(filtered_df.index, total=len(filtered_df)):

    model = SVC(probability=True,**literal_eval(filtered_df.at[i, 'best_rocauc_params']))
    
    dataset = filtered_df.at[i, 'dataset']
    target = filtered_df.at[i, 'target']
    FP = filtered_df.at[i, 'best_rocauc_FP']

    train_data = pd.read_csv(training_path+dataset+'/'+target+'.csv', header=0,index_col=False)
    X_train = batchECFP(train_data.iloc[:,2],FP)
    y_train = train_data.iloc[:,4]
    test_data = pd.read_csv(external_path+'ex_'+dataset+'/'+target+'.csv', header=0,index_col=False)
    X_test = batchECFP(test_data.iloc[:,1],FP)
    y_test = test_data.iloc[:,0]
    try:
        roc = run_ext(model,X_train,y_train,X_test,y_test,algorithm,dataset,target,FP)
    except Exception as e:
        roc = e
        print(e)
    filtered_df.at[i, 'external_roc'] = roc

filtered_df.to_excel(external_path+algorithm+'_ex_roc.xlsx',index=False)

  2%|▏         | 10/462 [00:07<05:34,  1.35it/s]

Only one class present in y_true. ROC AUC score is not defined in that case.


  3%|▎         | 13/462 [00:09<05:11,  1.44it/s]

Only one class present in y_true. ROC AUC score is not defined in that case.


  3%|▎         | 15/462 [00:11<05:22,  1.38it/s]

...

 98%|█████████▊| 452/462 [04:07<00:08,  1.22it/s]

Only one class present in y_true. ROC AUC score is not defined in that case.


 99%|█████████▉| 457/462 [04:14<00:05,  1.11s/it]

Only one class present in y_true. ROC AUC score is not defined in that case.


100%|██████████| 462/462 [04:19<00:00,  1.78it/s]


In [7]:
#更换算法时修改
import xgboost as xgb
algorithm = 'XGBoost'

filtered_df = training_list_df[training_list_df['model'] == algorithm]

for i in tqdm(filtered_df.index, total=len(filtered_df)):

    model = xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=1,**literal_eval(filtered_df.at[i, 'best_rocauc_params']))
    
    dataset = filtered_df.at[i, 'dataset']
    target = filtered_df.at[i, 'target']
    FP = filtered_df.at[i, 'best_rocauc_FP']

    train_data = pd.read_csv(training_path+dataset+'/'+target+'.csv', header=0,index_col=False)
    X_train = batchECFP(train_data.iloc[:,2],FP)
    y_train = train_data.iloc[:,4]
    test_data = pd.read_csv(external_path+'ex_'+dataset+'/'+target+'.csv', header=0,index_col=False)
    X_test = batchECFP(test_data.iloc[:,1],FP)
    y_test = test_data.iloc[:,0]
    try:
        roc = run_ext(model,X_train,y_train,X_test,y_test,algorithm,dataset,target,FP)
    except Exception as e:
        roc = e
        print(e)
    filtered_df.at[i, 'external_roc'] = roc

filtered_df.to_excel(external_path+algorithm+'_ex_roc.xlsx',index=False)

  2%|▏         | 10/462 [00:12<08:52,  1.18s/it]

Only one class present in y_true. ROC AUC score is not defined in that case.


  3%|▎         | 13/462 [00:15<08:01,  1.07s/it]

Only one class present in y_true. ROC AUC score is not defined in that case.


  3%|▎         | 15/462 [00:18<08:25,  1.13s/it]

...

 98%|█████████▊| 452/462 [06:31<00:08,  1.17it/s]

Only one class present in y_true. ROC AUC score is not defined in that case.


 99%|█████████▉| 457/462 [06:37<00:05,  1.08s/it]

Only one class present in y_true. ROC AUC score is not defined in that case.


100%|██████████| 462/462 [06:42<00:00,  1.15it/s]
